In [6]:
import gymnasium as gym
import os
#import gym
import numpy as np

import collections
import pickle
import tqdm

from stable_baselines3.common.buffers import ReplayBuffer


import os
import random
import time
from dataclasses import dataclass

import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import tyro
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter

from typing import Optional

import functools
import random
from copy import copy

import numpy as np
from gymnasium.spaces import Discrete, MultiDiscrete, Box, Dict

from pettingzoo import AECEnv



import gymnasium


from pettingzoo.utils import agent_selector, wrappers

from gymnasium.utils import EzPickle



from statistics import NormalDist

import pygame

from typing import Any , Generic, Iterable, Iterator, TypeVar
ActionType = TypeVar("ActionType")

import collections
from torch.utils.data import Dataset, DataLoader


C:\Users\31721\.conda\envs\DL_project\lib\site-packages\pkg_resources\__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
C:\Users\31721\.conda\envs\DL_project\lib\site-packages\pkg_resources\__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
C:\Users\31721\.conda\envs\DL_project\lib\site-packages\pkg_resources\__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-

In [7]:
from Utilities.new_models import *
import utils_gym
import env_model_class_2


from board_env import *



%load_ext autoreload
%autoreload 2

In [8]:
from IPython.display import clear_output, display

# hero agent definition

In [9]:
import numpy as np
class Hero_agent(int):
    def init_properties(self,agent_count,phases,cp=[],df=[],direct_action=True):
        #self.draw_count = 0
        self.init_win_count_iter(agent_count)
        self.init_move_count_epi(phases)
        self.cp = cp
        self.df = df
        self.direct_action = direct_action
        self.init_reward_concern(agent_count,cp=cp,df=df)
        
    def init_reward_concern(self,agent_count,cp=[],df=[]):
        if len(cp)==0:
            cp = [int(self)]
        self.concern=torch.tensor([(1 if i in cp 
                             else 
                             (-1 if i in df 
                                  else 0)) for i in range(1,agent_count+1) ])
        #self.concern_2 = self.concern
        #self.concern_2[self-1] =0
        
        self.multi_dependency = (sum(self.concern !=0)>1)
        
        
        

        
    def init_win_count_iter(self,agent_count):
        self.count_dict = {i:0 for i in range(1,agent_count+1)}
        self.count_draw_dict = {i:0 for i in range(1,agent_count+1)}
        self.draw_territory_count = 0
    def init_move_count_epi(self,phases):
        self.bad_move_count = 0
        self.bad_move_phase_count = {i:0 for i in phases}
        self.move_count =  {i:0 for i in phases}        
    
    def model_def(self, model):
        self.model =model

    def action_predict(self,data):
        return self.model.action_predict(data)
    def save_models(self):
        self.model.save_models()

    def process_reward(self,rewards,step,hero_steps):
        if self.multi_dependency and self.direct_action:
            return (rewards*self.concern.to(rewards.device)).sum(-1)[:step][hero_steps][:,None]
        elif self.multi_dependency and not self.direct_action:
            base_rew = torch.zeros( rewards[:step,self-1][hero_steps].shape)
            #print(base_rew)

            hero_step_list  = np.arange(0,step)[hero_steps]
            for i,j in zip(hero_step_list[:-1],hero_step_list[1:]):
                if j-i>1:
                    #print(j,i,rewards[i:j],(rewards[i:j]*self.concern),(rewards[i:j]*self.concern).sum())
                    base_rew[i]+= (rewards[i:j]*self.concern).sum()
            #print(base_rew,rewards[hero_step_list[-1]:],(rewards[hero_step_list[-1]:]*self.concern))
            base_rew[-1]+= (rewards[hero_step_list[-1]:]*self.concern).sum()
            
            return base_rew[:,None]
            
        else:
            return rewards[:step][hero_steps][:,None]
    
    #def model_forward_call(self,name,kwarg):
    #    return self.model_dict[name](**kwarg)
        

a = Hero_agent(1)
a.init_properties(3,[1,2,3],cp=[1],df=[2])

# dataset definition 
## episode trajectory
## dataloader for per iterations

In [971]:
import torch
from torch.utils.data import Dataset
from torch.nn.functional import pad

class TrajectoryDataset_per_episode(Dataset): #this should have only 1 trajectory no matter what
    def __init__(self, trajectories, context_len, rtg_scale,gamma=0.99,min_len = 10**6):
        self.trajectories = trajectories
        self.context_len = context_len

        min_len = min(min_len, len(self.trajectories[0]))
        
        #states = []
        #for traj in self.trajectories:
        #    traj_len = traj['observations'].shape[0]
        #    min_len = min(min_len, traj_len)
        #    states.append(traj['observations'])
        #    # calculate returns to go and rescale them 
            
        self.trajectories[0]['returns_to_go'] = discount_cumsum(self.trajectories[0]['rewards'], gamma) / rtg_scale
            
        #print(min_len)
        
        # used for input normalization
        
        #states = torch.concatenate(states, axis=0).to(dtype = torch.float32)
        self.state_mean, self.state_std = torch.mean(self.trajectories[0]['observations'], axis=0
                                                    ), torch.std(self.trajectories[0]['observations'], axis=0) + 1e-6

        # normalize states
        #for traj in self.trajectories:
        #    traj['observations'] = (traj['observations'].to(dtype=torch.float32) - self.state_mean) / self.state_std




    

    def __len__(self):
        #print(len(self.trajectories),(self.trajectories[0].shape),len(self.trajectories[0])- self.context_len + 1 )
        return sum(max(0, len(traj['observations'])- self.context_len + 1
                      ) for traj in self.trajectories)

    def __getitem__(self, idx):
        total_len = 0
        for traj in self.trajectories:
            
            #print(total_len, idx - total_len, total_len + len(traj), - self.context_len + 1)
            
            if total_len  <= idx < total_len + len(traj['observations']) - self.context_len + 1    :
                si = idx - total_len
                
                #context = traj[si:si + self.context_length]
                states = (traj['observations'][si : si + self.context_len])
                actions = (traj['actions'][si : si + self.context_len])
                reward =  (traj['rewards'][si : si + self.context_len])
                returns_to_go = (traj['returns_to_go'][si : si + self.context_len])
                
                action_masks = (traj['action_masks'][si : si + self.context_len])
                current_agent = (traj['current_agent'][si : si + self.context_len])
                current_agent_simple = (traj['current_agent_simple'][si : si + self.context_len])
                current_phase = (traj['current_phase'][si : si + self.context_len])
                current_troops_count = (traj['current_troops_count'][si : si + self.context_len])
    
                
                timesteps = torch.arange(start=si, end=si+self.context_len, step=1)
    
                # all ones since no padding
                traj_mask = torch.ones(self.context_len, dtype=torch.long)



                if self.context_len> len(states):
                    padding_len = self.context_len - len(states)
    
                    states                = torch.cat([states,
                                    torch.zeros(([padding_len] + list(states.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)  
                    actions               = torch.cat([actions,
                                    torch.zeros(([padding_len] + list(actions.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)   
                    reward                = torch.cat([reward,
                                    torch.zeros(([padding_len] + list(reward.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)  
                    returns_to_go         = torch.cat([returns_to_go,
                                    torch.zeros(([padding_len] + list(returns_to_go.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)         
                    action_masks          = torch.cat([action_masks,
                                    torch.zeros(([padding_len] + list(action_masks.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)        
                    current_agent         = torch.cat([current_agent,
                                    torch.zeros(([padding_len] + list(current_agent.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)
                    
                    current_troops_count = torch.cat([current_troops_count,
                                        torch.zeros(([padding_len] + list(current_troops_count.shape[1:])),
                                        dtype=current_troops_count.dtype
                                                   )], 
                                       dim=0)
                    current_phase         = torch.cat([current_phase,
                                    torch.zeros(([padding_len] + list(current_phase.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)         
                    current_troops_count  = torch.cat([current_troops_count,
                                    torch.zeros(([padding_len] + list(current_troops_count.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)                
                    traj_mask             = torch.cat([traj_mask,
                                    torch.zeros(([padding_len] + list(traj_mask.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)     














                
                return  timesteps, states, actions, returns_to_go,reward, traj_mask ,action_masks,current_agent_simple,current_agent,current_phase,current_troops_count
                
                #return pad(torch.tensor(context), (0,(self.context_length - len(context))),mode="constant"), [1]

            total_len += len(traj) - self.context_len + 1

        raise IndexError("Index out of range 1")


class TrajectoryDataset_2_through_episodes(Dataset):
    def __init__(self, trajectories):
        self.trajectories = trajectories

        all_obs = torch.concat([ traj.dataset.trajectories[0]['observations'] for traj in self.trajectories],axis=0)
        self.state_mean = torch.mean(all_obs,axis =0)
        self.state_std = torch.std(all_obs,axis =0) + 1e-6
        
        for traj in self.trajectories:
            traj.dataset.trajectories[0]['observations'] = (traj.dataset.trajectories[0]['observations'].to(dtype=torch.float32) - self.state_mean) / self.state_std


        #print(self.state_mean,self.state_std)
        
    def get_state_stats(self):
        return self.state_mean, self.state_std        

    

    def __len__(self):
        return len(self.trajectories)

    def __getitem__(self, idx):
        total_len = 0
        if total_len  <= idx < total_len + len(self.trajectories)  :
            return [batch for batch in self.trajectories[idx] ][0]


        raise IndexError("Index out of range 2")

# custom sampler for the hero agent 
(but will not be used)

In [683]:
type(T.paths[0].dataset[0])

tuple

In [687]:
from torch.utils.data.sampler import  Sampler

class YourSampler(Sampler[list[int]]):
    def __init__(self, mask,data_len):
        self.mask = mask[:,None]
        self.indices = np.arange(data_len)

    def __iter__(self):

        return (self.indices[i] for i in self.mask)

    def __len__(self):
        return len(self.mask)


sampler1 = YourSampler(torch.tensor([2,3]),4)

trainloader_sampler1 = torch.utils.data.DataLoader([([1],[2]),([5],[2]),([6],[3]),([7],[4])], batch_size=4,
                                          sampler = sampler1, shuffle=False)
[batch for batch in trainloader_sampler1]

[[[tensor([6, 7])], [tensor([3, 4])]]]

# old dataset and definition

In [973]:

class D4RLTrajectoryDataset(Dataset):
    def __init__(self, trajectories, context_len, rtg_scale,gamma=0.99,min_len = 10**6):

        self.context_len = context_len
        self.trajectories = trajectories

        #print(len(self.trajectories))
        # load dataset
        #with open(dataset_path, 'rb') as f:
        #    self.trajectories = pickle.load(f)

        
        # calculate min len of traj, state mean and variance
        # and returns_to_go for all traj

        
        
        states = []
        for traj in self.trajectories:
            traj_len = traj['observations'].shape[0]
            min_len = min(min_len, traj_len)
            states.append(traj['observations'])
            # calculate returns to go and rescale them
            
            traj['returns_to_go'] = discount_cumsum(traj['rewards'], gamma) / rtg_scale
            
        print(min_len)
        
        # used for input normalization
        states = torch.concatenate(states, axis=0).to(dtype = torch.float32)
        self.state_mean, self.state_std = torch.mean(states, axis=0), torch.std(states, axis=0) + 1e-6

        # normalize states
        #for traj in self.trajectories:
        #    traj['observations'] = (traj['observations'].to(dtype=torch.float32) - self.state_mean) / self.state_std


    def get_state_stats(self):
        return self.state_mean, self.state_std

    def __len__(self):
        #print(len(self.trajectories))
        return len(self.trajectories)

    def __getitem__(self, idx):
        traj = self.trajectories[idx]
        traj_len = traj['observations'].shape[0]

        if traj_len >= self.context_len:
            # sample random index to slice trajectory
            si = random.randint(0, traj_len - self.context_len)

            states = (traj['observations'][si : si + self.context_len])
            actions = (traj['actions'][si : si + self.context_len])
            reward =  (traj['rewards'][si : si + self.context_len])
            returns_to_go = (traj['returns_to_go'][si : si + self.context_len])
            
            action_masks = (traj['action_masks'][si : si + self.context_len])
            current_agent_simple = (traj['current_agent_simple'][si : si + self.context_len])
            current_agent = (traj['current_agent'][si : si + self.context_len])
            current_phase = (traj['current_phase'][si : si + self.context_len])
            current_troops_count = (traj['current_troops_count'][si : si + self.context_len])

            
            timesteps = torch.arange(start=si, end=si+self.context_len, step=1)

            # all ones since no padding
            traj_mask = torch.ones(self.context_len, dtype=torch.long)

        else:
            padding_len = self.context_len - traj_len

            # padding with zeros
            states = (traj['observations'])
            states = torch.cat([states,
                                torch.zeros(([padding_len] + list(states.shape[1:])),
                                dtype=states.dtype)], 
                               dim=0)
            
            actions = (traj['actions'])
            actions = torch.cat([actions,
                                torch.zeros(([padding_len] + list(actions.shape[1:])),
                                dtype=actions.dtype)], 
                               dim=0)
            reward = (traj['rewards'])
            reward = torch.cat([reward,
                                torch.zeros(([padding_len] + list(reward.shape[1:])),
                                dtype=reward.dtype
                                           )], 
                               dim=0)
            returns_to_go = (traj['returns_to_go'])
            returns_to_go = torch.cat([returns_to_go,
                                torch.zeros(([padding_len] + list(returns_to_go.shape[1:])),
                                dtype=returns_to_go.dtype
                                           )], 
                               dim=0)

            action_masks = (traj['action_masks'][si : si + self.context_len])
            current_agent_simple = (traj['current_agent_simple'][si : si + self.context_len])
            current_agent = (traj['current_agent'][si : si + self.context_len])
            current_phase = (traj['current_phase'][si : si + self.context_len])
            current_troops_count = (traj['current_troops_count'][si : si + self.context_len])

            action_masks = torch.cat([action_masks,
                                torch.zeros(([padding_len] + list(action_masks.shape[1:])),
                                dtype=action_masks.dtype
                                           )], 
                               dim=0)
            current_agent_simple = torch.cat([current_agent_simple,
                                torch.zeros(([padding_len] + list(current_agent_simple.shape[1:])),
                                dtype=current_agent_simple.dtype
                                           )], 
                               dim=0)
            current_agent = torch.cat([current_agent,
                                torch.zeros(([padding_len] + list(current_agent.shape[1:])),
                                dtype=current_agent.dtype
                                           )], 
                               dim=0)

            current_phase = torch.cat([current_phase,
                                torch.zeros(([padding_len] + list(current_phase.shape[1:])),
                                dtype=current_phase.dtype
                                           )], 
                               dim=0)

            current_troops_count = torch.cat([current_troops_count,
                                torch.zeros(([padding_len] + list(current_troops_count.shape[1:])),
                                dtype=current_troops_count.dtype
                                           )], 
                               dim=0)



            
            timesteps = torch.arange(start=0, end=self.context_len, step=1)

            traj_mask = torch.cat([torch.ones(traj_len, dtype=torch.long), 
                                   torch.zeros(padding_len, dtype=torch.long)], 
                                  dim=0)
            
        return  timesteps, states, actions, returns_to_go,reward, traj_mask ,action_masks,current_agent_simple,current_agent,current_phase,current_troops_count




#  Algorithm module definition 

## DDQN

In [500]:
class DDQN_module:
    def __init__(self, qnet_config_dict, actor_config_dict,args,device,writer,run_name,agent):

        self.agent = agent
        self.run_name =run_name 
        self.actor_config_dict = actor_config_dict
        self.qnet_config_dict = qnet_config_dict
        self.args = args
        self.device = device
        self.writer = writer
        self.entropy=args.entropy
        self.return_prob=args.return_prob
        self.actor_wt = args.actor_wt
        self.CE_wt = args.CE_wt
        if self.args.small:
            self.actor = Actor_ddqn_small(**self.actor_config_dict).to(self.device)
            self.qf1 = QNetwork_small(**self.qnet_config_dict).to(self.device)
            self.qf1_target = QNetwork_small(**self.qnet_config_dict).to(self.device)
            self.target_actor = Actor_ddqn_small(**self.actor_config_dict).to(self.device)
            
        else:
            self.actor = Actor_ddqn(**self.actor_config_dict).to(self.device)
            self.qf1 = QNetwork(**self.qnet_config_dict).to(self.device)
            self.qf1_target = QNetwork(**self.qnet_config_dict).to(self.device)
            self.target_actor = Actor_ddqn(**self.actor_config_dict).to(self.device)
            
            
        
        
        self.target_actor.load_state_dict(self.actor.state_dict())
        self.qf1_target.load_state_dict(self.qf1.state_dict())
        self.q_optimizer = optim.Adam(list(self.qf1.parameters()), lr=self.args.learning_rate)
        self.actor_optimizer = optim.Adam(list(self.actor.parameters()), lr=self.args.learning_rate)

    def action_predict(self,data):
        return self.actor(data)

    def train_write(self,data,iteration,epoch):
        #data = rb.sample(self.args.batch_size)
        qf1_a_values, qf1_loss, actor_loss = self.train(data)
        self.write(qf1_a_values, qf1_loss, actor_loss,epoch,iteration)

    def cal_q_loss(self,qf1_a_values, next_q_value):
        criterion = nn.SmoothL1Loss()
        
        return criterion(qf1_a_values, next_q_value)
        #return nn.functional.mse_loss(qf1_a_values, next_q_value)

    
    def cal_actor_loss(self,data,entropy=False,return_prob=False):

        if not entropy:
            return -self.qf1(data.observations[:,:-1], self.actor(data.observations[:,:-1])).mean()
        else:
            actions,probs = self.actor(data.observations[:,:-1],return_prob=return_prob)
            action_loss = -self.qf1(data.observations[:,:-1],actions).mean()

            entropy_ = -(probs*torch.log2(probs)).sum(-1).mean()
            return self.actor_wt*action_loss + self.CE_wt*entropy_# if not torch.isnan(cross_entropy_loss) else 0)
        

    def train(self,data):


        
        with torch.no_grad():
            collected_t_next = data.next_observations[:,-1]
            next_state_actions = self.target_actor(data.next_observations[:,:-1])
            qf1_next_target = self.qf1_target(data.next_observations[:,:-1], next_state_actions)

            
            next_q_value = data.rewards.flatten() + (1 - data.dones.flatten()) * (self.args.gamma**(collected_t_next+1)).view(-1) * (qf1_next_target).view(-1)
    
        qf1_a_values = self.qf1(data.observations[:,:-1], data.actions).view(-1)
        qf1_loss = self.cal_q_loss(qf1_a_values, next_q_value)
        
        # optimize the model
        self.q_optimizer.zero_grad()
        qf1_loss.backward()
        self.q_optimizer.step()
        
        #if global_step % args.policy_frequency == 0:
        actor_loss = self.cal_actor_loss(data,entropy=self.entropy,return_prob=self.return_prob)
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
    
        # update the target network
        for param, target_param in zip(self.actor.parameters(), self.target_actor.parameters()):
            target_param.data.copy_(self.args.tau * param.data + (1 - self.args.tau) * target_param.data)
        for param, target_param in zip(self.qf1.parameters(), self.qf1_target.parameters()):
            target_param.data.copy_(self.args.tau * param.data + (1 - self.args.tau) * target_param.data)
        return qf1_a_values, qf1_loss, actor_loss
        

    def write(self,qf1_a_values, qf1_loss, actor_loss,epoch,iteration):
        
        ind_epoch = epoch + (iteration-1)*self.args.update_epochs
        self.writer.add_scalar(f"losses/{self.agent}/qf1_values", qf1_a_values.mean().item(), ind_epoch)
        
        self.writer.add_scalar(f"losses/{self.agent}/qf1_loss", qf1_loss.item(), ind_epoch)
        self.writer.add_scalar(f"losses/{self.agent}/actor_loss", actor_loss.item(), ind_epoch)
        
    def save_models(self):
        newpath = r'./models/'+ self.run_name +'/'+str(self.agent)
        if not os.path.exists(newpath):
            os.makedirs(newpath)
        torch.save(self.actor.state_dict(), newpath+"/actor.pt")
        torch.save(self.qf1.state_dict(), newpath+"/qf1.pt")
        torch.save(self.qf1_target.state_dict(), newpath+"/qf1_target.pt")
        torch.save(self.target_actor.state_dict(), newpath+"/target_actor.pt")       
    

    def load_models(self):
        newpath = r'./models/'+ self.run_name +'/'+str(self.agent)
        self.actor.load_state_dict(torch.load(newpath+"/actor.pt"))
        self.qf1.load_state_dict(torch.load(newpath+"/qf1.pt"))
        self.qf1_target.load_state_dict(torch.load(newpath+"/qf1_target.pt"))
        self.target_actor.load_state_dict(torch.load(newpath+"/target_actor.pt")) 
        

                                          
                   

## transfomer

In [ ]:
class transformer_model:
    def __init__(self,hero = 1):
        self.hero = hero
        state_dim = env.observation_space.shape[0]
        act_dim = env.action_space.n #3#1 #env.action_space.shape[0]
        #context_len_=200
        model = DecisionTransformer(
                    
                    state_dim=state_dim,
                    act_dim=act_dim,
                    n_blocks=n_blocks,
                    h_dim=embed_dim,
                    context_len=context_len,
                    n_heads=n_heads,
                    drop_p=dropout_p,
                ).to(device)
          
        optimizer_2 = torch.optim.AdamW(
                            model.parameters(), 
                            lr=0.000005,#lr, 
                            weight_decay=wt_decay
                        )
        
        
        #lr = 0.00001
        optimizer_1 = torch.optim.AdamW(
                            model.parameters(), 
                            lr=lr,#lr, 
                            weight_decay=wt_decay
                        )
        
        scheduler = torch.optim.lr_scheduler.LambdaLR(
                optimizer_1,
                lambda steps: min((steps+1)/warmup_steps, 1)
            )
        
        #scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_1, T_0=150, T_mult=2, eta_min=0.01, last_epoch=-1)
        
        max_d4rl_score = -1000.0
        total_updates = 0
        
        
        if training_division :
            freeze_non_value_layers(model,value=False)
            value_cal_loss = False
        else:
            value_cal_loss = True
    def action_loss_fn(self,action_preds_2,action_mask,return_preds_2,returns_target,beta = 0.2):
        #return nn.CrossEntropyLoss().forward(action_preds_2,action_target)
        returns_to_go_actor = action_mask*returns_target[:,None]
    
        loss = (F.softmax( action_preds_2,-1)*((beta*(1 - action_mask) + action_mask)*torch.pow(returns_to_go_actor - action_preds_2,2).detach())).mean()
        return loss
    
    def value_loss_fn(self,return_preds,returns_target,action_mask,beta=0.2):
        returns_to_go_actor = action_mask*returns_target[:,None]
        loss = ((beta*(1 - action_mask) + action_mask)*torch.pow(returns_to_go_actor - return_preds,2)).mean()
        
        return loss#F.mse_loss(return_preds, returns_to_go_actor, reduction='mean')
    
    def value_loss_fn_2(self,return_preds,return_preds_2,returns_target,
                        action_preds_2,reward,action_target,
                        beta=0.2,alpha=1,gamma =0.99,device=device):
    
        part_1 =  alpha*(torch.log(torch.exp(return_preds_2).sum(axis =-1)) - (action_mask*return_preds_2).sum(axis =-1) ).mean()
    
        c_ = (F.one_hot(action_target).reshape(action_preds.shape)*return_preds).sum(-1)[:,:,None].to(device)
        c = torch.roll(c_, shifts=-1, dims=-2).to(device)
        c[:,-1,:]=0 # this is an issue when context length is not equal to the episode len, .... maybe make use of target mask? and add a sudo 
    
        #print(reward[:,:-1,None].device, c[:,:-1,:].device, return_preds[:,:-1,:].device)
        
        part_2 = 0.5*torch.pow(reward[:,:-1,None] + gamma*c[:,:-1,:] - return_preds[:,:-1,:],2).mean()
        loss =  part_2 + part_1 #+ 
        #returns_to_go_actor = action_mask*returns_target[:,None]
        #loss = ((beta*(1 - action_mask) + action_mask)*torch.pow(returns_to_go_actor - return_preds,2)).mean()
        #print(part_1,part_2)
        return loss
    


    
    def calculate_loss(self,timesteps,
                            action_target,
                            returns_target,
                            state_preds, 
                            action_preds,
                            return_preds,
                            reward,
                            returns_to_go):
        
        # only consider non padded elements
        action_preds_2 = action_preds.view(-1, act_dim)[traj_mask.view(-1,) > 0].to(device)
        action_target = action_target.squeeze().view(-1)[traj_mask.view(-1,) > 0].to(device)
        return_preds_2 = return_preds.squeeze().view(-1, act_dim)[traj_mask.view(-1,) > 0].to(device)
        return_target = returns_to_go.squeeze().view(-1)[traj_mask.view(-1,) > 0].to(device)
        action_mask = F.one_hot(action_target).squeeze().to(device)
        policy_loss = nn.CrossEntropyLoss().forward(action_preds_2,action_target)#action_loss_fn(action_preds_2,action_mask,return_preds_2,return_target,beta = 1)
        
        if value_cal_loss:
            Q_loss = self.value_loss_fn_2(return_preds,
                                     return_preds_2,
                                     returns_target,
                                     action_preds_2,
                                     reward,
                                     action_target,
                                     beta=0.2,alpha=2,gamma =0.99)#value_loss_fn(return_preds_2,return_target,action_mask,beta=1)
            total_loss = Q_loss+policy_loss
        else:
            total_loss = policy_loss

        return total_loss


    
    def train_write(self,data):
        
        timesteps, states, actions, returns_to_go,reward, traj_mask ,action_masks,current_agent_simple,current_agent,current_phase,current_troops_count = data

    

        
        timesteps = timesteps[0].to(self.device)	# B x T
        states = states[0].to(self.device)			# B x T x state_dim
        actions = actions[0].to(self.device)		# B x T x act_dim
        reward = reward[0].to(self.device)
        returns_to_go = returns_to_go[0].to(self.device).unsqueeze(dim=-1) # B x T x 1
        traj_mask = traj_mask[0].to(self.device)	# B x T
        action_masks =action_masks[0].to(self.device)
        current_agent_simple = current_agent_simple[0].to(self.device)
        current_agent =current_agent[0].to(self.device)
        current_phase =current_phase[0].to(self.device)
        current_troops_count =current_troops_count[0].to(self.device)

        
        info = dict({})
        
        hero_steps = (current_agent_simple == self.hero )
        
        states = torch.concat(
            
                
                (states,current_phase,current_agent
            
            ,torch.ones(len(action_masks))[:,None]*self.hero
            ,current_troops_count[:,self.hero,None]
            ,action_masks*hero_steps[:,None]), axis =1)[0] 
                
            

        action_target,returns_target = torch.clone(actions).detach().to(device),torch.clone(actions).detach().to(device)
        
        state_preds, action_preds, return_preds = model.forward(
                                                        timesteps=timesteps,
                                                        states=states,
                                                        actions=actions,
                                                        returns_to_go=returns_to_go
                                                        ,info = info
            

            
                                                    )
        


        

        total_loss = self.calculate_loss(timesteps,
                                   action_target,
                                   returns_target,
                                   state_preds, 
                                   action_preds,
                                   return_preds,
                                   reward,
                                   returns_to_go)


        #action_loss = #nn.CrossEntropyLoss().forward(action_preds_2,action_target)#F.mse_loss(action_preds_2, action_target.float(), reduction='mean')
        optimizer_1.zero_grad()
        #policy_loss.backward(retain_graph=True)
        (total_loss).backward()   
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        optimizer_1.step()
        scheduler.step()

        #if value_cal_loss:
        #    q_los = Q_loss.detach().cpu().item()
        #    writer.add_scalar("Q_loss", q_los, iyr+i_train_iter*num_updates_per_iter)
        #    log_Q_losses.append(q_los)
        #
        #pol_los = policy_loss.detach().cpu().item()
        #writer.add_scalar("Policy_loss", pol_los, iyr+i_train_iter*num_updates_per_iter)
        #log_action_losses.append(pol_los)
                
            
                
            	## evaluate on env   
                #results = evaluate_on_env(model, device, context_len, env, rtg_target, rtg_scale,
            	#                        num_eval_ep, max_eval_ep_len, state_mean, state_std, 
            	#						)
            #
                #writer.add_scalar("eval/avg_reward", results['eval/avg_reward'], i_train_iter)
                #writer.add_scalar("eval/avg_ep_len", results['eval/avg_ep_len'], i_train_iter)
                #writer.add_scalar("eval/'total_reward",results['total_reward'],i_train_iter)
            #
            #
                #
                #eval_avg_reward = results['eval/avg_reward']
                #eval_avg_ep_len = results['eval/avg_ep_len']
                #eval_d4rl_score = results['eval/avg_reward']#get_d4rl_normalized_score(results['eval/avg_reward'], env_name) * 100
                #mean_action_loss,mean_Q_loss = np.mean(log_action_losses),np.mean(log_Q_losses)
                #
                #writer.add_scalar("mean_action_loss_per_iter", mean_action_loss, i_train_iter)
                #writer.add_scalar("mean_Q_loss_per_iter", mean_Q_loss, i_train_iter)
            #
            #
                #
                #time_elapsed = str(datetime.now().replace(microsecond=0) - start_time)
                #total_updates += num_updates_per_iter
                #log_str = ("=" * 60 + '\n' +
            	#		"time elapsed: " + time_elapsed  + '\n' +
            	#		"num of updates: " + str(total_updates) + '\n' +
            	#		"action loss: " +  format(mean_action_loss, ".5f") + '\n' +
            	#		"Q loss: " +  format(mean_Q_loss, ".5f") + '\n' +
            	#		"eval avg reward: " + format(eval_avg_reward, ".5f") + '\n' +
            	#		"eval avg ep len: " + format(eval_avg_ep_len, ".5f") + '\n' +
            	#		"eval d4rl score: " + format(eval_d4rl_score, ".5f") +'\n'+
                #        " iterations: "+str(i_train_iter) +" calulate_value_loss :" +str(value_cal_loss)
            	#		)
            #
                #print(log_str)
            #
                #log_data = [time_elapsed, total_updates, mean_action_loss,mean_Q_loss,
            	#			eval_avg_reward, eval_avg_ep_len,
            	#			eval_d4rl_score]
            #
                #csv_writer.writerow(log_data)
            	#
            	## save model
                #print("max d4rl score: " + format(max_d4rl_score, ".5f"))
                #if eval_d4rl_score >= max_d4rl_score:
                #    print("saving max d4rl score model at: " + save_best_model_path)
                #    torch.save(model.state_dict(), save_best_model_path)
                #    max_d4rl_score = eval_d4rl_score
            #
                #print("saving current model at: " + save_model_path)
                #torch.save(model.state_dict(), save_model_path)
                
            #writer.add_hparams(dict(exp_name=log_csv_name,env_name=env_name,state_dim=state_dim,
            #			act_dim=int(act_dim),
            #			n_blocks=n_blocks,
            #			h_dim=embed_dim,
            #			context_len=context_len,
            #			n_heads=n_heads,
            #			drop_p=dropout_p,training_division=training_division,
            #            start_value_training_iteration=start_value_training_iteration,optim=optimizer_1.__class__.__name__,lr=lr,
            #                       scheduler=scheduler.__class__.__name__, sch_param = ",".join([ i+":"+str(j) for i,j in dict(T_0=150, T_mult=2, eta_min=0.01, last_epoch=-1).items()])
            #                       ,batch_size=batch_size),{'hparam/eval_avg_reward':eval_avg_reward})            
            
            #print("=" * 60)
            #print("finished training!")
            #print("=" * 60)
            #end_time = datetime.now().replace(microsecond=0)
            #time_elapsed = str(end_time - start_time)
            #end_time_str = end_time.strftime("%y-%m-%d-%H-%M-%S")
            #print("started training at: " + start_time_str)
            #print("finished training at: " + end_time_str)
            #print("total training time: " + time_elapsed)
            #print("max d4rl score: " + format(max_d4rl_score, ".5f"))
            #print("saved max d4rl score model at: " + save_best_model_path)
            #print("saved last updated model at: " + save_model_path)
            #print("=" * 60)
        
            

In [881]:
hero = 1
(((T.paths[0]).dataset.trajectories[0]['current_agent']*torch.tensor([[1,2,3]])).sum(-1) == (hero-1) )

tensor([ True,  True, False,  ..., False, False, False])

In [736]:
action_mask = (T.paths[0]).dataset.trajectories[0]['action_masks']*hero_steps[:,None]

In [742]:
((T.paths[0]).dataset.trajectories[0]).keys()

dict_keys(['observations', 'next_observations', 'rewards', 'terminals', 'actions', 'action_masks', 'current_agent', 'current_phase', 'current_troops_count', 'returns_to_go'])

In [891]:
(timesteps, states, actions, returns_to_go,reward, traj_mask ,
             action_masks,current_agent,current_phase,current_troops_count) = [i for i in T.traj_data_loader ][0]

In [947]:
states[0].shape

torch.Size([1601, 256, 10, 5])

In [945]:
states[0].shape[:-2]

torch.Size([1601, 256])

In [959]:
states[0].reshape(list(states[0].shape[:-2]) + [np.prod(states[0].shape[-2:])])[0][0]

tensor([ 0.0000, -0.6905, -1.3072,  0.0000, -1.0107, -0.0668, -1.0696, -0.7206,
        -0.2330, -0.7806, -0.2660, -0.7691, -0.7559, -0.4148, -0.9676,  0.0000,
        -0.7129, -0.8337, -0.5691, -1.2656, -0.4469, -0.6254, -1.0183,  0.0000,
        -1.0231, -0.5956, -0.6374, -0.8425, -0.0758, -1.0404, -0.5329, -0.5539,
        -0.9112,  0.0000, -1.3050, -0.5437, -0.6443, -0.8929,  0.0000, -1.3344,
         0.0000, -1.2906, -0.6843,  0.0000, -1.2122,  0.0000, -2.4390, -0.3955,
         0.0000, -0.7329])

In [967]:
torch.concat((states[0].reshape(list(states[0].shape[:-2]) + [np.prod(states[0].shape[-2:])]),
              
              actions[0]),axis =-1).shape


torch.Size([1601, 256, 52])

In [885]:

hero = 1
hero_steps = (((T.paths[0]).dataset.trajectories[0]['current_agent']*torch.tensor([[1,2,3]])).sum(-1) == (hero-1) )

torch.concat(
    (
        torch.concat(
                    (
                T.paths[0].dataset.trajectories[0]['observations'].reshape(-1,np.prod(T.ob_space_shape))
                ,T.paths[0].dataset.trajectories[0]['current_phase']
                ,T.paths[0].dataset.trajectories[0]['current_agent']), axis =1)

,torch.ones(len(T.paths[0].dataset.trajectories[0]['action_masks']))[:,None]*hero
,T.paths[0].dataset.trajectories[0]['current_troops_count'][:,hero,None]
,T.paths[0].dataset.trajectories[0]['action_masks']*hero_steps[:,None]), axis =1)[0]


tensor([ 0.0000, -0.6905, -1.3072,  0.0000, -1.0107, -0.0668, -1.0696, -0.7206,
        -0.2330, -0.7806, -0.2660, -0.7691, -0.7559, -0.4148, -0.9676,  0.0000,
        -0.7129, -0.8337, -0.5691, -1.2656, -0.4469, -0.6254, -1.0183,  0.0000,
        -1.0231, -0.5956, -0.6374, -0.8425, -0.0758, -1.0404, -0.5329, -0.5539,
        -0.9112,  0.0000, -1.3050, -0.5437, -0.6443, -0.8929,  0.0000, -1.3344,
         0.0000, -1.2906, -0.6843,  0.0000, -1.2122,  0.0000, -2.4390, -0.3955,
         0.0000, -0.7329,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,
         3.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
         1.0000,  1.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,
         1.0000])

In [780]:
(T.paths[0]).dataset.trajectories[0]['observations'].reshape(-1,np.prod(T.ob_space_shape))
torch.ones(len((T.paths[0]).dataset.trajectories[0]['action_masks']))*1
(T.paths[0]).dataset.trajectories[0]['action_masks']*hero_steps[:,None]
((T.paths[0]).dataset.trajectories[0])['current_phase']
((T.paths[0]).dataset.trajectories[0])['current_agent']
((T.paths[0]).dataset.trajectories[0])['current_troops_count'][:,1]


((T.paths[0]).dataset.trajectories[0])['returns_to_go'][:,1]
((T.paths[0]).dataset.trajectories[0])['actions'][:]*hero_steps[:,None]

((T.paths[0]).dataset.trajectories[0])['rewards'][:,1]#*hero_steps[:]


tensor([3., 3., 3.,  ..., 3., 3., 3.])

In [ ]:
hero_steps = [current_agent == agent] # agent = hero_agent

b_obs_a_all = torch.concat(( obs.reshape(-1,np.prod(self.ob_space_shape)) ,
                            action_masks.reshape(-1,np.prod(self.action_mask_shape)),
                            self.map_agent_phase_vector(current_agent,num_classes=self.total_agents+1)[:,1:],
                            
                            #map_agent_phase_vector(current_agent,num_classes=len(env.possible_agents)),
                            self.map_agent_phase_vector(current_phase,num_classes=self.total_phases)#,
                            #current_troops_count[:,self.the_hero_agent-1,None],
                            #selected_t_next
                           ),axis =1)



In [ ]:


            

            t_range = torch.Tensor(np.arange(0,step)).to(self.device,dtype=torch.int)
            hero_steps = [current_agent == self.the_hero_agent][0][:,0][:step]
            
            next_indecies = (t_next[:step,self.the_hero_agent-1].to(dtype=torch.int) + t_range +1)[hero_steps].long()
        
            selected_t_next = t_next[:,self.the_hero_agent-1,None]#t_next[:step,0,None][hero_steps]
            infos = [dir({})]*step #t_next[:step,0]
            b_obs_a_all = torch.concat(( obs.reshape(-1,np.prod(self.ob_space_shape)) ,
                                    action_masks.reshape(-1,np.prod(self.action_mask_shape)),
                                    self.map_agent_phase_vector(current_agent,num_classes=self.total_agents+1)[:,1:],
                                    
                                    #map_agent_phase_vector(current_agent,num_classes=len(env.possible_agents)),
                                    self.map_agent_phase_vector(current_phase,num_classes=self.total_phases)#,
                                    #current_troops_count[:,self.the_hero_agent-1,None],
                                    #selected_t_next
                                   ),axis =1)



        avg_episode_length = torch.mean(torch.tensor(
                            [(episodes[:step] == i_epi).sum() for i_epi in episodes[:step].unique()]).float())#np.mean([(episodes[:step] == i_epi).sum() for i_epi in episodes[:step].unique()])
        if self.args.TB_log:
            self.writer.add_scalar("charts/avg_episodic_length", avg_episode_length, self.global_step)



        return paths
        #return rb

    def update_hero_rb(self,b_obs_a_all,t_next,t_range,current_agent,actions,
                                rewards,dones,infos,current_troops_count,
                                step):



            for agent in self.hero_agents_list:
                #the_hero_agent = agent
                
                
                hero_steps = [current_agent == agent][0][:,0][:step]
                next_indecies = (t_next[:step,agent-1].to(dtype=torch.int) + t_range +1)[hero_steps].long()
                selected_t_next = t_next[:,agent-1,None]#t_next[:step,0,None][hero_steps]

                b_obs_a = torch.concat((b_obs_a_all,
                                        current_troops_count[:,agent-1,None],
                                        selected_t_next),axis =1)
                
                for i in zip(b_obs_a[:step][hero_steps].cpu().to(dtype=torch.float), b_obs_a[next_indecies].cpu().to(dtype=torch.float), 
                   actions[:step][hero_steps].cpu().to(dtype=torch.float32), rewards[:step][hero_steps][:,agent-1,None].cpu(), 
                   dones[:step][hero_steps][:,agent-1,None].cpu(), infos):
                    self.hero_agents_list[agent].rb.add(*i)
        

# trainer module

In [502]:








class Trainer:
    """
    ## Trainer
    """

    def __init__(self, Args,param_dict =dict({})
                 ):
        # #### Configurations

        self.args = Args()#tyro.cli(Args)
        self.param_dict = param_dict
        self.update_arg(param_dict=param_dict)
        self.device = self.args.device#torch.device("cuda" if torch.cuda.is_available() and self.args.cuda else "cpu")
        
        
        #self.args.batch_size = int(self.args.num_envs * self.args.num_steps)
        self.args.minibatch_size = int(self.args.batch_size // self.args.num_minibatches)
        #self.args.num_iterations = self.args.total_timesteps // self.args.batch_size
        self.gam = self.args.gamma
        #self.args.minibatch_size = 256#128 
        self.num_steps = self.args.num_steps#120000#1000000
        self.num_iterations = self.args.num_iterations
        self.episode_time_lim = self.args.episode_time_lim
        self.hero_agent_count = self.args.hero_agent_count
        self.env_config = self.args.env_config
        self.num_episodes = self.args.num_episodes
        self.context_len=200
        
        #self.env_config = dict(render_mode = 'rgb_array', default_attack_all  = True,
        #                    agent_count  = 4
        #                       ,use_placement_perc=True,render_=False)        
        
        self.run_name = f"{self.args.env_id}__{self.args.exp_name}__{self.args.seed}__{int(time.time())}"

        

        TB_log = self.args.TB_log 
        if TB_log:    
            self.writer = SummaryWriter(f"runs/{self.run_name}")
            self.writer.add_text(
                "hyperparameters",
                "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(self.args).items()])),
            )
        else:
            self.writer = None
        
        # TRY NOT TO MODIFY: seeding
        random.seed(self.args.seed)
        np.random.seed(self.args.seed)
        #torch.manual_seed(self.args.seed)
        
        torch.backends.cudnn.deterministic = self.args.torch_deterministic
        
        
        
        self.playe_r = 1#"agent_1" #
        

        
        self.action_shape = (2,)



        self.env = env_risk(**self.env_config)
        
        self.env.reset(seed=42)


            

        
        self.total_agents  = len(self.env.possible_agents)
        self.total_phases = len(self.env.phases)
        #print(self.env.last()[0]['observation'])

        print(torch.tensor(self.env.last()[0]['observation']
                          ,device=self.device))
        sample_obs = self.obs_converter(torch.tensor(self.env.last()[0]['observation']
                                                    ).to(device=self.device),
                                        num_classes = self.total_agents+1
                                       )
        
        self.ob_space_shape = sample_obs.shape #env.observation_space(playe_r)['observation'].shape
        self.action_mask_shape = self.env.observation_space(self.playe_r)['action_mask'].shape
        
        #self.device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")

        
        
        self.agent_list = list(self.env.possible_agents)
        
        
        
        
       
        self.the_hero_agent = 1

        
        self.qnet_config_dict = dict(action_space = self.env.action_space(self.playe_r
                                                                         ).shape[0],
                                    ob_space=np.prod(self.ob_space_shape
                                                    )+np.prod(self.action_mask_shape)
                                         +1*( self.total_agents -1) #the current_agent +1#who actor agent was
                                         +1*(self.total_phases -1)#the current phase
                                         +1 # the number of troops
                               )
        self.actor_config_dict =  dict(env=self.env,
                        action_space = self.env.observation_space(self.playe_r)['action_mask'].shape[0],
                        ob_space=np.prod(self.ob_space_shape)+np.prod(self.action_mask_shape)
                                         +1*( self.total_agents-1) #the current_agent +1#who actor agent was
                                         +1*(self.total_phases -1)#the current phase
                                         +1 # the number of troops
                               )
        
        
        self.hero_agents_list = {i:Hero_agent(i) for i in range(1,self.hero_agent_count+1) } # this is a list , need to pass it as an argument
        
        for i in self.hero_agents_list:
            self.hero_agents_list[i].init_properties(self.total_agents,self.env.phases)        
        
            self.hero_agents_list[i].model_def(model = DDQN_module( self.qnet_config_dict, self.actor_config_dict,
                                                                   self.args,device = self.device,writer=self.writer,run_name=self.run_name,agent=i)
                                                )
            

            #self.target_actor.load_state_dict(self.actor.state_dict())
            #self.qf1_target.load_state_dict(self.qf1.state_dict())
            #self.q_optimizer = optim.Adam(list(self.qf1.parameters()), lr=self.args.learning_rate)
            #self.actor_optimizer = optim.Adam(list(self.actor.parameters()), lr=self.args.learning_rate)

    def update_arg(self,param_dict=dict({})):
       for i,j in param_dict.items():
           setattr(self.args,i,j)

        

    def obs_converter(self,  data, num_classes = 4, col =0 ):

        if col != None:

            #print(data.device)
            #print(nn.functional.one_hot(data[:4,col].detach().long(), 
            #                                            num_classes = num_classes).to(self.device))
            return torch.concat((nn.functional.one_hot(data[:,col].detach().long(), 
                                                        num_classes = num_classes).to(self.device),
                                      data[:,~col,None]
                                ),axis=1
                               )[:,1:].to(self.device)
    
    def map_agent_phase_hot(self, data,num_classes = 3):
        return nn.functional.one_hot(torch.tensor(data),num_classes = num_classes)[1:].to(self.device)
    
    def map_agent_phase_vector(self, data,num_classes = 3):
        return nn.functional.one_hot(data[:,0].long(), 
                                                            num_classes = num_classes)[:,1:].to(self.device)


    def train_loop_init(self):
        self.gamma_t = {i:0 for i in self.env.possible_agents}
        
        
        self.draw_count = 0

        for i in self.hero_agents_list:
            self.hero_agents_list[i].init_win_count_iter(self.total_agents )
        
        
        #self.first_count = 0
        #self.second_count = 0
        #self.third_count = 0
        #self.third_count_draw = 0
        
        self.start_time = time.time()
        self.global_step = 0
        #self.faulting_player = ""

    
    
    def run_training_loop(self):
        """
        ### Run training loop
        """

        # last 100 episode information
        #tracker.set_queue('reward', 100, True)
        #tracker.set_queue('length', 100, True)

        obs = torch.zeros((self.num_steps,) + self.ob_space_shape).to(self.device)
        actions = torch.zeros((self.num_steps, ) + self.action_shape).to(self.device)
        action_masks = torch.zeros((self.num_steps, ) + self.action_mask_shape).to(self.device)
        current_agent = torch.ones((self.num_steps,1)).to(self.device)*0#-1
        current_phase = torch.zeros((self.num_steps,1)).to(self.device)
        current_troops_count = torch.zeros((self.num_steps,self.total_agents)).to(self.device)
        logprobs = torch.zeros((self.num_steps, )).to(self.device)
        rewards = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
        rewards_2 = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
        dones = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
        values = torch.zeros((self.num_steps,  )).to(self.device)
        episodes = torch.ones((self.num_steps, )).to(self.device)*-1
        t_next = torch.zeros((self.num_steps, self.total_agents)).to(self.device)


        for i in self.hero_agents_list: # each agent has his own buffer, this is kinda pain because now this information is stored and not discarded
    
            self.hero_agents_list[i].rb = ReplayBuffer(
                    self.args.buffer_size,
                    Box(low =0, high=2000, shape =(self.qnet_config_dict['ob_space']+1,), dtype=np.float32),
                    Box(low =0, high=2000, shape =(2,), dtype=np.float32),
                    self.device,
                    handle_timeout_termination=False,
                )

        env = env_risk(**(self.env_config #| {"render_mode" : None, "bad_mov_penalization" : 0.01,"render_":False}
                         ))
        env.reset(42)
        
        self.train_loop_init()
        self.paths=[]
        
        for iteration in range(1, self.num_iterations+1):

            obs = torch.zeros((self.num_steps,) + self.ob_space_shape).to(self.device)
            actions = torch.zeros((self.num_steps, ) + self.action_shape).to(self.device)
            action_masks = torch.zeros((self.num_steps, ) + self.action_mask_shape).to(self.device)
            current_agent = torch.ones((self.num_steps,1)).to(self.device)*0#-1
            current_phase = torch.zeros((self.num_steps,1)).to(self.device)
            current_troops_count = torch.zeros((self.num_steps,self.total_agents)).to(self.device)
            logprobs = torch.zeros((self.num_steps, )).to(self.device)
            rewards = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
            rewards_2 = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
            dones = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
            dones_2 = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
            #values = torch.zeros((self.num_steps,  )).to(self.device)
            episodes = torch.ones((self.num_steps, )).to(self.device)*-1
            t_next = torch.zeros((self.num_steps, self.total_agents)).to(self.device)








            
            self.paths = self.sample(
                                env,iteration,
                                obs
                                ,actions
                                ,action_masks
                                ,current_agent
                                ,current_phase
                                ,current_troops_count
                                ,logprobs
                                ,rewards
                                ,rewards_2
                                ,dones
                                ,dones_2
                                ,values
                                ,episodes
                                ,t_next
                                
                 
                            )

            

            #self.traj_dataset = D4RLTrajectoryDataset(self.paths, context_len=self.args.context_len,
            #                                             rtg_scale=self.args.rtg_scale,gamma=self.args.gamma)
            #                                            #rtg_scale=1,gamma=0.99
    
            #traj_dataset = traj_dataset.to(self.args.pin_memory_device)
            
            #self.traj_data_loader = DataLoader(self.traj_dataset,
            #						batch_size=self.args.batch_size,
            #                        shuffle= self.args.shuffle,
            #                        pin_memory= self.args.pin_memory,
            #                        drop_last = self.args.drop_last,
            #                        pin_memory_device=self.args.pin_memory_device
            #                        #shuffle=True,
            #                        #pin_memory=True,
            #                        #drop_last=True
            #                        )

            self.traj_dataset = TrajectoryDataset_2_through_episodes(self.paths ) # a dataset of dataloaders
        
            self.traj_data_loader = DataLoader(self.traj_dataset, batch_size=1,shuffle=self.args.shuffle,
                                    pin_memory= self.args.pin_memory,
                                    drop_last = self.args.drop_last,
                                    pin_memory_device=self.args.pin_memory_device) # only spit 1 episode a time


            
            
            #self.data_iter = iter(self.traj_data_loader)
            
            ## get state stats from dataset
            #state_mean, state_std = self.traj_dataset.get_state_stats()

            #print(len(self.paths))
            #print(len(traj_dataset))
            
            #print(len(self.data_iter))
            #print(next(data_iter))
            #(timesteps, states, actions, returns_to_go,reward, traj_mask ,
            # action_masks,current_agent,current_phase,current_troops_count) = next(self.data_iter)

            #for batch in self.traj_data_loader

            #print(states.shape)
            
            break




            
            self.train(iteration)


            
            
            #if self.global_step%100 ==0:
            #    SPS = int(self.global_step / (time.time() - self.start_time))
            #    print("SPS:", SPS)       
            #    self.writer.add_scalar("charts/SPS", SPS, self.global_step)
        
            
            self.save_models()

    def train(self,iteration):
        for epoch in range(self.args.update_epochs):
            
            if self.global_step > self.args.learning_starts:

                for i in self.hero_agents_list:
                    for batch in self.traj_data_loader
                    self.hero_agents_list[i].model.train_write(batch,iteration,epoch)
                    
    
    def train_(self,iteration):
        
        for epoch in range(self.args.update_epochs):
            
            if self.global_step > self.args.learning_starts:

                for i in self.hero_agents_list:
                    self.hero_agents_list[i].model.train_write(self.hero_agents_list[i].rb.sample(self.args.batch_size)
                                                         ,iteration,epoch)
                    
    def save_models(self):
        for i in self.hero_agents_list:
            self.hero_agents_list[i].save_models()  

    def reset_moves_hero_agents(self):
        for i in self.hero_agents_list:
            self.hero_agents_list[i].init_move_count_epi(self.env.phases)
    
    def sample(self,env,iteration,obs
                                ,actions
                                ,action_masks
                                ,current_agent
                                ,current_phase
                                ,current_troops_count
                                ,logprobs
                                ,rewards
                                ,rewards_2
                                ,dones
                                ,dones_2
                                ,values
                                ,episodes
                                ,t_next
                                ):
        

        paths = []
        
        with torch.no_grad():
            # sample `worker_steps` from each worker
            #there are no worker steps... rather there are full episodes

            step = 0
            fault_condition = False
            clear_output(wait=True)
            phase = 0
        



            

            
            for episode in range(self.num_episodes):#num_episodes):
                
                total_rewards = {i:0 for i in env.possible_agents} #i can report this
                action=1
                
                if fault_condition:
                    env = env_risk(**(self.env_config  #| {"render_mode" : None,"bad_mov_penalization" : 0.01,"render_":False#False
                                                        # }
                                     )
                                      )#game.env(render_mode=None)

                curren_epi = episode + (iteration-1)*self.num_episodes
                env.reset(curren_epi) #for riplication
                
                fault_condition = False
                step_count = 0
                
                self.reset_moves_hero_agents()
                is_draw = 0
                
                #draw_territory_count = 0
                #is_third = 0

                for agent in env.agent_iter():
                    e_t = env.terminations
                    if sum(e_t.values()) <(self.total_agents-1):
                        observation, reward, termination, truncation, info = env.last()
        
                        observation['observation'] =  self.obs_converter(
                                                        torch.tensor(
                                                            observation['observation']
                                                        ).to(self.device,dtype=torch.float32),
                                                        num_classes = self.total_agents+1)
                        
                        episodes[step] = curren_epi
                        obs[step] = observation['observation']#torch.Tensor(observation['observation']).to(self.device) #sould i not add it .... meaning this is the last observation after the player dies
                        action_masks[step] = torch.Tensor(observation['action_mask']).to(self.device)
                        
                        #curr_agent = agent#int(agent[-1])
                        current_agent[step] = curr_agent = agent
                        current_phase[step] = phase = env.phase_selection
                        phase_mapping = self.map_agent_phase_hot(phase,num_classes = self.total_phases).float()
                        
                        curr_agent_mapping = self.map_agent_phase_hot(int(curr_agent)-1,
                                                                      num_classes = self.total_agents 
                                                                     ).float()
                        
                        current_troops_count[step] = torch.Tensor([env.board.agents[i].bucket for i in env.possible_agents]).to(self.device)
                    

                        model_in = torch.Tensor(torch.hstack((observation['observation'].reshape(-1),torch.tensor(observation['action_mask'].reshape(-1)).to(self.device),
                                           phase_mapping,
                                            curr_agent_mapping,
                                           torch.tensor([env.board.agents[curr_agent].bucket ]).to(self.device)))[None,:]#.repeat(3,axis = 0)
                                                ).float()
                        
                        #if e_t[curr_agent]:
                            #print('heeee')
                            
                        if termination or truncation: #this never happens ... the agent is removed from the current agent list and processed after the end of the cycle
                            
                            action = None

                            act = self.actor(torch.Tensor(model_in).to(self.device))
                            #act = self.
                            #act, logprob, _, value = agent_mod.get_action_and_value(model_in)
                            #values[step] = value.flatten() # so even if we are removing the guy ... we need to know what is the action he would 
                                                                #have taken at this point and what would have been its value
                            actions[step] = act #even after going what would have been
                            #logprobs[step] = logprob        
                        else:
                            mask = observation["action_mask"]
                            if (self.global_step < self.args.learning_starts) or (
                                np.random.rand() > min(
                                                ((curren_epi)/((self.num_iterations*self.num_episodes)/10))
                                                , 0.95)
                                                #) or (agent != self.the_hero_agent) 
                                                )or ( agent not in self.hero_agents_list):
        
                                
                                action = env.action_space(agent).sample()
                                #part_0 =np.random.choice(np.where(env.board.calculated_action_mask(agent))[0])
                                part_0 =np.random.choice(np.where(observation['action_mask'])[0])
                                action = torch.Tensor([[[part_0],[np.around(action[1],2)]]]).to(self.device)
                                action = action[:,:,0]
                            else:
                                action = self.hero_agents_list[curr_agent].action_predict(torch.Tensor(model_in).to(self.device))
                                #action = self.actor(torch.Tensor(model_in).to(self.device))
                            actions[step] = action
                            curr_agent_ = int(curr_agent)
        
                            if not observation['action_mask'][action[:,0].long()]: 
                                fault_condition =True
                                #self.faulting_player = agent

                                


                                if  curr_agent_ in self.hero_agents_list:
                                    self.hero_agents_list[curr_agent_].bad_move_count+=1
                                    self.hero_agents_list[curr_agent_].bad_move_phase_count[int(current_phase[step][0])]+=1  # when is the where_is_it_performing_bad_really
                                    #print('here',agent, action, observation['action_mask'])
                            
        
                            if  curr_agent_ in self.hero_agents_list:
                                self.hero_agents_list[curr_agent_].move_count[int(current_phase[step][0])]+=1  
                            #if self.the_hero_agent == curr_agent:
                                #move_count[int(current_phase[step][0])]+=1        
        
        
                        #print('here',agent, action)
                        if action != None :
                            act_2 = action.detach().cpu().numpy()[0]#list([action.detach().cpu().numpy()[0][0], max(action.detach().cpu().numpy()[0][1],0.1) ])
                            act_2 = list([act_2[0], max(act_2[1],0.001) ])
                        else:
                            act_2 = action
                            
                        env.step(act_2 if action != None else None)        
        
        
                        if action == None:
                            print('heeee')
                            rewards[step] = np.zeros(self.total_agents) # should i keep it -1? .... hm i dont think so .
                            dones[step] = np.zeros(self.total_agents) # frankly the guys is already done so we really dont have to do anything here.... this is the state post termination for a loser 
                            # but btw this is for the next agent ... action == None means in the last action the previous agent would have been removed.
                            #values[step] = 
                        else:
        
                            
                            curr_reward_list =  env.curr_rewards
                            
                            if (step_count == (self.episode_time_lim-1)): # draw reward
                                is_draw=1
                                curr_reward_list = {i:-100 for i in env.possible_agents }

                            rewards_2[step] = torch.Tensor([curr_reward_list[i] for i in env.possible_agents]).to(self.device)
                            if step >1:
                                dones_2[step] = torch.Tensor([ int(env.terminations[i]) - dones_2[step-1,i-1]  for i in env.possible_agents]).to(self.device)
                            else:
                                dones_2[step] = torch.Tensor([env.terminations[i] for i in env.possible_agents]).to(self.device)
                                
                            for i in env.possible_agents:
                                if i != curr_agent:
                                    self.gamma_t[i]+=1
                                else:
                                    self.gamma_t[i] =0
        
                                if (step_count == (self.episode_time_lim-1)):
                                    cr_rew = -100
                                    term = True
                                else:
                                    cr_rew = env.curr_rewards[i]
                                    term = env.terminations[i]

                                next_step_ = step-self.gamma_t[i]
                                rewards[next_step_,i-1] += (self.args.gamma**self.gamma_t[i])*cr_rew
                                t_next[next_step_,i-1] = self.gamma_t[i]
                                dones[next_step_,i-1] = torch.Tensor([term]).to(self.device) #so the panetly has to be added but attributions is really difficult
        
                        #list_curr_reward_list = np.array(list(curr_reward_list.values()))
                        
                        #if sum(curr_reward_list.values()) == -300:
                            #print('here')
                            #is_draw=1
        
                        
                        for age_i in env.possible_agents:
                            
                            total_rewards[age_i]+=curr_reward_list[age_i] #env.curr_rewards[age_i] if (step_count != episode_time_lim) else -100
                                    
                        
                        step +=1
                        self.global_step+=1
        
                    else:
                        print('done:',env.terminations,#env.terminations.values(),
                              ",total_reward:",total_rewards, ',iteration:',iteration,",episode:", episode )
                        break    
                
        
        
        
                    step_count+=1
                    
                    if (self.global_step == self.num_steps) :# or (fault_condition and (fa ulting_player != agent) and (len(env.agents)==0)):
                        break
                    elif (step_count == self.episode_time_lim):
                        break
                        
                #print(rewards[step-2])
                if self.global_step == self.num_steps:
                    break 

                for i in self.hero_agents_list:
                    self.hero_agents_list[i].position =self.total_agents
                    
                #[ position = 3 for i in ] 
                for k_,(i_,j_) in enumerate(sorted([(j_,i_) for i_,j_ in total_rewards.items()],reverse=True) 
                      ):
                    if int(j_) in self.hero_agents_list:
                        self.hero_agents_list[int(j_)].position = k_+1
                        
                        
                    #if j_==self.the_hero_agent:
                    #    position = k_+1

                cur_epi_list = (episodes == curren_epi)
                if self.args.TB_log:
                    self.write_exploring(is_draw,#position,
                            curren_epi,step,
                            total_rewards,#bad_move_count
                            #,bad_move_phase_count,
                            #move_count,
                            observation,
                            env,
                            cur_epi_list)

                #paths = []

                data_ = collections.defaultdict(torch.tensor)
                data_['observations'] = obs[:step_count].to(device =self.args.pin_memory_device).reshape(-1,np.prod(T.ob_space_shape))
                #data_['next_observations'] = obs[1:step_count+1].to(device =self.args.pin_memory_device).reshape(-1,np.prod(T.ob_space_shape)) #torch.tensor([1,2,3,4])
                data_['rewards'] = rewards_2[:step_count].to(device =self.args.pin_memory_device)#torch.tensor([1,2,3,4])
                data_['terminals'] = dones_2[:step_count].to(device =self.args.pin_memory_device)#torch.tensor([1,2,3,4])
                data_['actions'] = actions[:step_count].to(device =self.args.pin_memory_device)#torch.tensor([1,2,3,4])
                data_['action_masks'] = action_masks[:step_count].to(device =self.args.pin_memory_device)
                data_['current_agent_simple'] = current_agent[:step_count].to(device =self.args.pin_memory_device)
                data_['current_agent'] = self.map_agent_phase_vector(current_agent[:step_count],num_classes=self.total_agents+1)[:,1:].to(device =self.args.pin_memory_device)
                data_['current_phase'] = self.map_agent_phase_vector(current_phase[:step_count],num_classes=self.total_phases).to(device =self.args.pin_memory_device)
                data_['current_troops_count'] = current_troops_count[:step_count].to(device =self.args.pin_memory_device)
                
                #data_['state'] = torch.concat(
                #                    (
                #                data_['observations'].reshape(-1,np.prod(T.ob_space_shape))
                #                ,data_['current_phase']
                #                ,data_['current_agent']), axis =1)
                #hero = 1
                #hero_steps = (((T.paths[0]).dataset.trajectories[0]['current_agent']*torch.tensor([[1,2,3]])).sum(-1) == (hero-1) )
                
                #torch.concat(
                #    (
                #        torch.concat(
                #                    (
                #                T.paths[0].dataset.trajectories[0]['observations'].reshape(-1,np.prod(T.ob_space_shape))
                #                ,T.paths[0].dataset.trajectories[0]['current_phase']
                #                ,T.paths[0].dataset.trajectories[0]['current_agent']), axis =1)
                #
                #,torch.ones(len(T.paths[0].dataset.trajectories[0]['action_masks']))[:,None]*hero
                #,T.paths[0].dataset.trajectories[0]['current_troops_count'][:,hero,None]
                #,T.paths[0].dataset.trajectories[0]['action_masks']*hero_steps[:,None]), axis =1)[0]
                
                

                
                datase = TrajectoryDataset_per_episode([data_],context_len=self.args.context_len,
                                                         rtg_scale=self.args.rtg_scale,gamma=self.args.gamma)
                paths.append(DataLoader(datase, batch_size=len(datase)))

                    # Example usage:
        #trajectories_1 = T.paths[:1]
        
        #trajectories_2 = T.paths[1:2]
        #dataset_1 = TrajectoryDataset_per_episode(trajectories_1,rtg_scale=1, context_len=200)
        #dataset_2 = TrajectoryDataset_per_episode(trajectories_2,rtg_scale=1, context_len=200)
        
        
        
        #dataset = TrajectoryDataset(trajectories, context_length=5)
        #dataloader_1 = DataLoader(dataset_1, batch_size=len(dataset_1))
        #dataloader_2 = DataLoader(dataset_2, batch_size=len(dataset_2))
        #dataset_3_ = TrajectoryDataset_2_through_episodes([dataloader_1,dataloader_2])
        
        #dataloader_3 = DataLoader(dataset_3_, batch_size=1,shuffle=True)
        
        # Iterate through the dataloader
        
        #torch.concat([batch for batch in dataloader])
        #for batch in dataloader_3:
        #    print(batch[1][0].shape)








            



            

            t_range = torch.Tensor(np.arange(0,step)).to(self.device,dtype=torch.int)
            hero_steps = [current_agent == self.the_hero_agent][0][:,0][:step]
            
            next_indecies = (t_next[:step,self.the_hero_agent-1].to(dtype=torch.int) + t_range +1)[hero_steps].long()
        
            selected_t_next = t_next[:,self.the_hero_agent-1,None]#t_next[:step,0,None][hero_steps]
            infos = [dir({})]*step #t_next[:step,0]
            b_obs_a_all = torch.concat(( obs.reshape(-1,np.prod(self.ob_space_shape)) ,
                                    action_masks.reshape(-1,np.prod(self.action_mask_shape)),
                                    self.map_agent_phase_vector(current_agent,num_classes=self.total_agents+1)[:,1:],
                                    
                                    #map_agent_phase_vector(current_agent,num_classes=len(env.possible_agents)),
                                    self.map_agent_phase_vector(current_phase,num_classes=self.total_phases)#,
                                    #current_troops_count[:,self.the_hero_agent-1,None],
                                    #selected_t_next
                                   ),axis =1)

            #self.update_hero_rb(b_obs_a_all,t_next,t_range,current_agent,actions,
            #                    rewards,dones,infos,current_troops_count,
            #                    step)

            
            #for i in zip(b_obs_a[:step][hero_steps].cpu().to(dtype=torch.float), b_obs_a[next_indecies].cpu().to(dtype=torch.float), 
            #   actions[:step][hero_steps].cpu().to(dtype=torch.float32), rewards[:step][hero_steps][:,self.the_hero_agent-1,None].cpu(), 
            #   dones[:step][hero_steps][:,self.the_hero_agent-1,None].cpu(), infos):
            #    rb.add(*i)




        avg_episode_length = torch.mean(torch.tensor(
                            [(episodes[:step] == i_epi).sum() for i_epi in episodes[:step].unique()]).float())#np.mean([(episodes[:step] == i_epi).sum() for i_epi in episodes[:step].unique()])
        if self.args.TB_log:
            self.writer.add_scalar("charts/avg_episodic_length", avg_episode_length, self.global_step)



        return paths
        #return rb

    def update_hero_rb(self,b_obs_a_all,t_next,t_range,current_agent,actions,
                                rewards,dones,infos,current_troops_count,
                                step):



            for agent in self.hero_agents_list:
                #the_hero_agent = agent
                
                
                hero_steps = [current_agent == agent][0][:,0][:step]
                next_indecies = (t_next[:step,agent-1].to(dtype=torch.int) + t_range +1)[hero_steps].long()
                selected_t_next = t_next[:,agent-1,None]#t_next[:step,0,None][hero_steps]

                b_obs_a = torch.concat((b_obs_a_all,
                                        current_troops_count[:,agent-1,None],
                                        selected_t_next),axis =1)
                
                for i in zip(b_obs_a[:step][hero_steps].cpu().to(dtype=torch.float), 
                             b_obs_a[next_indecies].cpu().to(dtype=torch.float), 
                            actions[:step][hero_steps].cpu().to(dtype=torch.float32),
                             rewards[:step][hero_steps][:,agent-1,None].cpu(), 
                           dones[:step][hero_steps][:,agent-1,None].cpu(), infos):
                            self.hero_agents_list[agent].rb.add(*i)
        



        
        

    def write_exploring(self,is_draw,#position,
                        curren_epi,step,
                        total_rewards,#bad_move_count
                        #,bad_move_phase_count,
                        #move_count,
                        observation,
                        env,
                        cur_epi_list):

        if is_draw:
            self.draw_count +=1

            for i in self.hero_agents_list:

                self.writer.add_scalar(f"draw_charts_agent_{i}/position_draw",self.hero_agents_list[i].position
                                                                                            ,self.draw_count)




                
                self.hero_agents_list[i].draw_territory_count = int(observation['observation'][:,i].sum())
                
                self.hero_agents_list[i].count_draw_dict[
                                                        self.hero_agents_list[i].position
                                                        ] +=1
                self.writer.add_scalar(f"draw_charts_agent_{i}/draw_territory_count",
                                                                               self.hero_agents_list[i].draw_territory_count,
                                                                               self.draw_count)#self.global_step)
                
                for j in self.hero_agents_list[i].count_draw_dict:
                    self.writer.add_scalar(f"draw_charts_agent_{i}/{j}_position_prop_draw",int(
                                                                                            self.hero_agents_list[i].position==j
                                                                                            ),self.draw_count)

                    if j not in [1,2]:
                        self.writer.add_scalar(f"win_charts_agent_{i}/{j}_position_all_prop",int(
                                                                                            self.hero_agents_list[i].position==j
                                                                                            ),(curren_epi+1))

                        self.writer.add_scalar(f"draw_charts_agent_{i}/{j}_place_in_draw",
                                                                   self.hero_agents_list[i].count_draw_dict[j],
                                                                   self.draw_count)

                        self.writer.add_scalar(f"draw_charts_agent_{i}/{j}_place_in_draw_ratio",
                                                                   self.hero_agents_list[i].count_draw_dict[j]/self.draw_count,
                                                                   self.draw_count)
                        
                    
            self.writer.add_scalar("draw_charts/draw_count",self.draw_count,self.global_step)
            self.writer.add_scalar("draw_charts/draw",1,(curren_epi+1))
            self.writer.add_scalar("draw_charts/draw_to_total_count",self.draw_count/(curren_epi +1+0.000001),self.global_step)
            
        else:
            
            non_draw_count =(curren_epi-self.draw_count+1+0.000001)
            for i in self.hero_agents_list:

                self.writer.add_scalar(f"win_charts_agent_{i}/position_win",self.hero_agents_list[i].position
                                                                                            ,(curren_epi+1))



                
                self.hero_agents_list[i].count_dict[self.hero_agents_list[i].position
                                               ] +=1

                
                for j in self.hero_agents_list[i].count_dict:
                    self.writer.add_scalar(f"win_charts_agent_{i}/{j}_position_prop",int(
                                                            self.hero_agents_list[i].position==j
                                                            ),(curren_epi+1))

                    self.writer.add_scalar(f"win_charts_agent_{i}/{j}_position",self.hero_agents_list[i].count_dict[j],
                                           (curren_epi+1))

                    self.writer.add_scalar(f"win_charts_agent_{i}/{j}_position_to_total_terminated",self.hero_agents_list[i].count_dict[j]/non_draw_count,(curren_epi+1))

                    if j not in [1,2]:
                        self.writer.add_scalar(f"win_charts_agent_{i}/{j}_position_all_prop",int(
                                                                                            self.hero_agents_list[i].position==j
                                                                                            ),(curren_epi+1))


            self.writer.add_scalar("draw_charts/draw",0,(curren_epi+1))
            
        for i in self.hero_agents_list:

            self.writer.add_scalar(f"win_charts_agent_{i}/position_all",self.hero_agents_list[i].position
                                                                                            ,(curren_epi+1))

            
            for j in self.hero_agents_list[i].count_dict:
                self.writer.add_scalar(f"win_charts_agent_{i}/{j}_position_to_total",(
                                                            self.hero_agents_list[i].count_dict[j]+
                                                            self.hero_agents_list[i].count_draw_dict[j]
                                                        
                                                            )/(curren_epi +1+0.00001 ),(curren_epi+1))#global_step)

            
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_count_per_episode",
                                               self.hero_agents_list[i].bad_move_count,self.global_step)
                
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_count_position_per_episode",
                                               self.hero_agents_list[i].bad_move_phase_count[0],self.global_step)
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_count_attack_per_episode",
                                               self.hero_agents_list[i].bad_move_phase_count[1],self.global_step)
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_count_fortify_per_episode",
                                               self.hero_agents_list[i].bad_move_phase_count[2],self.global_step)
            
            self.writer.add_scalar(f"moves/agent_{i}/total_moves",sum(
                                                    self.hero_agents_list[i].move_count.values()),self.global_step)
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_to_step_count_per_episode",
                                               self.hero_agents_list[i].bad_move_count/(sum(
                                                   self.hero_agents_list[i].move_count.values())+1),self.global_step)
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_to_step_position_per_episode",
                                               self.hero_agents_list[i].bad_move_phase_count[0]/( 
                                                   self.hero_agents_list[i].move_count[0]+1),self.global_step)
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_to_step_attack_per_episode",
                                               self.hero_agents_list[i].bad_move_phase_count[1]/( 
                                                   self.hero_agents_list[i].move_count[1]+1),self.global_step)
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_to_step_fortify_per_episode",
                                               self.hero_agents_list[i].bad_move_phase_count[2]/( 
                                                   self.hero_agents_list[i].move_count[2]+1),self.global_step)

        
        self.writer.add_scalar("charts/epsilon",(curren_epi/((self.num_iterations*self.num_episodes)/10)),self.global_step)
        self.writer.add_scalar("charts/avg_per_epi_total_reward", np.mean(list(total_rewards.values())), self.global_step)

        

        #values_total = {i:0 for i in self.env.possible_agents}
        

        
        self.writer.add_scalar("charts/episodic_length", cur_epi_list[:step].sum(), self.global_step)
        
        for i in env.possible_agents:
            #cur_index = torch.where((current_agent[:,0] == i) &( cur_epi_list ))[0]

            #values_total[i] = values[cur_index].mean()
            #writer.add_scalar("charts/mean_value_per_epi_agent_"+str(i), values_total[i], global_step)
            
            self.writer.add_scalar("charts/total_reward_per_epi_agent_"+str(i), total_rewards[i], self.global_step)

    
    


        
        




In [35]:
type(data_)

collections.defaultdict

In [485]:
def discount_cumsum(x, gamma,Torch = True):
    if Torch:
        disc_cumsum = torch.zeros_like(x,dtype = torch.float32)
    else:
        disc_cumsum = np.zeros_like(x,dtype = np.float32)
    
    disc_cumsum[-1] = x[-1]
    #print(disc_cumsum[-1])
    for t in reversed(range(x.shape[0]-1)):
        disc_cumsum[t] = x[t] + gamma * disc_cumsum[t+1]
        #print(x[t],disc_cumsum[t])
    return disc_cumsum

In [486]:

exp_3 = dict(
exp_name = 'exp3_ddqn_lr_bs_1',
learning_rate = 0.001,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 2,
entropy=False,
return_prob=False,
small = False,
rtg_scale=1,
shuffle=True,
pin_memory=True,
drop_last=True,
num_episodes = 10,
    
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 4
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.01
                 
                     #| {"render_mode" : None,"bad_mov_penalization" : 0.01,"render_":False#False
                     #                                   }
                 
                 )

)

exp_4 = dict(
exp_name = 'exp4_ddqn_2_agents_1_hero',
learning_rate = 0.001,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 1,
entropy=False,
return_prob=False,
small = False,
rtg_scale=1,
shuffle=True,
pin_memory=True,
drop_last=True,
num_episodes = 10,
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 2
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.01
                 )

)


exp_5 = dict(
exp_name = 'exp5_ddqn_2_agents_1_hero',
learning_rate = 0.001,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 1,
entropy=False,
return_prob=False,
small = False,
rtg_scale=1,
shuffle=True,
pin_memory=True,
drop_last=True,
num_episodes = 10,
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 2
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.1
                 )

)


exp_6 = dict(
exp_name = 'exp6_ddqn_2_agents_1_hero',
learning_rate = 0.0003,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 1,
entropy=False,
return_prob=False,
small = False,
rtg_scale=1,
shuffle=True,
pin_memory=True,
drop_last=True,
num_episodes = 10,
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 2
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.1
                 )

)


exp_7 = dict(
exp_name = 'exp7_ddqn_2_agents_1_hero',
learning_rate = 0.0003,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 1,
entropy=True,
return_prob=2,
actor_wt = 0.5,
CE_wt = 0.01,
small = False,
rtg_scale=1,
shuffle=True,
pin_memory=True,
drop_last=True,
num_episodes = 10,
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 2
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.1
                 )

)

exp_8 = dict(
exp_name = 'exp8_ddqn_2_agents_1_hero_norm',
learning_rate = 0.0003,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 1,
entropy=True,
return_prob=2,
actor_wt = 0.5,
CE_wt = 0.01,
small = False,
rtg_scale=1,
shuffle=True,
pin_memory=True,
drop_last=True,
num_episodes = 10,
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 2
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.1
                 )

)

exp_9 = dict(
exp_name = 'exp9_ddqn_4_agents_2_hero_norm',
learning_rate = 0.0003,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 2,
entropy=True,
return_prob=2,
actor_wt = 0.5,
CE_wt = 0.01,
small = False,
rtg_scale=1,
shuffle=True,
pin_memory=True,
drop_last=True,
num_episodes = 10,
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 4
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.1
                 )

)




exp_10 = dict(
exp_name = 'exp10_ddqn_4_agents_2_hero_norm_small',
learning_rate = 0.0003,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 2,
entropy=True,
return_prob=2,
actor_wt = 0.5,
CE_wt = 0.01,
small = True,
rtg_scale=1,
shuffle=True,
pin_memory=True,
drop_last=True,
num_episodes = 10,
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 4
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.01
                 )

)


exp_11 = dict(
exp_name = 'exp11_ddqn_4_agents_2_hero_norm_small',
learning_rate = 0.0003,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 2,
entropy=True,
return_prob=2,
actor_wt = 0.5,
CE_wt = 0.01,
small = True,

context_len = 64,
rtg_scale=1,
shuffle=True,
pin_memory=True,
drop_last=True,
num_episodes = 5,
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 4
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.01
                 )

)



exp_12 = dict(
exp_name = 'exp12_ddqn_4_agents_2_hero_norm_small',
learning_rate = 0.0003,
batch_size = 2,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 2,
entropy=True,
return_prob=2,
actor_wt = 0.5,
CE_wt = 0.01,
small = True,
num_episodes = 5,
context_len = 256,
rtg_scale=1,
shuffle=True,
pin_memory=False,#False,
drop_last=True,
TB_log=False,
device =torch.device("cuda" if torch.cuda.is_available() else "cpu"),
pin_memory_device= "cpu",#("cuda" if torch.cuda.is_available() else "cpu"),

    
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 4
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.01
                 )

)

In [488]:
#T = Trainer(Args,param_dict = exp_3)
T = Trainer(Args,param_dict = exp_12)

tensor([[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]], device='cuda:0', dtype=torch.int8)


In [491]:
T.run_training_loop()

done: {1: True, 2: True, 3: True, 4: True} ,total_reward: {1: -200.06, 2: 106.34000000000002, 3: -97.58999999999999, 4: -300.02} ,iteration: 1 ,episode: 0
done: {1: True, 2: True, 3: True, 4: True} ,total_reward: {1: 108.73, 2: -97.38999999999999, 3: -300.03, 4: -200.01999999999998} ,iteration: 1 ,episode: 1
done: {1: True, 2: True, 3: True, 4: True} ,total_reward: {1: -300.01, 2: 108.9, 3: -98.14999999999999, 4: -199.01999999999998} ,iteration: 1 ,episode: 2
done: {1: True, 2: True, 3: True, 4: True} ,total_reward: {1: -98.17999999999999, 2: -199.04, 3: 108.68, 4: -300.0} ,iteration: 1 ,episode: 3
done: {1: True, 2: True, 3: True, 4: True} ,total_reward: {1: -300.01, 2: -99.1, 3: 109.9, 4: -199.07999999999998} ,iteration: 1 ,episode: 4
tensor([[0.0000e+00, 3.2287e-01, 6.3086e-01, 0.0000e+00, 3.1852e+00],
        [4.4490e-03, 5.3362e-01, 3.4181e-01, 5.1481e-02, 1.9830e+00],
        [6.6099e-02, 3.7168e-01, 3.6367e-01, 1.4682e-01, 2.9017e+00],
        [0.0000e+00, 3.3698e-01, 4.1007e-01

AttributeError: 'Trainer' object has no attribute 'traj_dataset'

In [89]:
env_config = dict(render_mode = 'rgb_array', default_attack_all  = True,
                render_ = True,agent_count  = 3,use_placement_perc=True)
env = env_risk(**(env_config | {"render_mode" : None,"verbose":False, "agent_count" :4,"render_":False}))


total_rewards = {i:0 for i in env.possible_agents}

step =0
for i__ in range(1):
    env.reset()#seed=42)
    #print(env_1.infos.keys())
    observation, reward, termination, truncation, info = env.last()
    
    total_rewards = {i:0 for i in env.possible_agents}
    
    #so there is an issue ...reward =2 sometimes ... doubling?

    for agent in env.agent_iter():
        step+=1
        e_t = env.terminations
        if sum(e_t.values()) <3:
            observation, reward, termination, truncation, info = env.last()
            
            action = env.action_space(agent).sample()
            
            #if env_1.phase_selection ==1:
            #    action = [int(action[0]),action[1]]
            part_0 =np.random.choice(np.where(env.board.calculated_action_mask(agent))[0])
            #print(part_0)
            
            #action = [part_0,np.around(action[1],2) #min(action[1],env_1.board.agents[agent].bucket ) 
            #                  if env_1.phase_selection==0 else  action[1]]
    
            action = [part_0,np.around(action[1],2)]
    
    
            for i in env.possible_agents:
                total_rewards[i]+=env.curr_rewards[i]
            
            env.step(action)

            if step%10000 ==0:
                print(step)
                print(env.board.territories)
                
                print('\naction',action,
                      '\ninfo',info,
                      '\ninfos',env.infos,
                      '\naction_valid',observation['action_mask'][action[0]],
                      '\nagent',agent,
                      '\nselected_agent',env.agent_selection,
                      '\ncurr_agent',env.board.current_agent,
                      '\ncurr phase',env.phase_selection,
                      '\nbad_trail count',env.board.bad_trials,
                      '\nmax_bad trails', env.board.max_bad_trials,
                      '\nreward',reward,
                      '\nreward',env.curr_rewards,
                      '\ntotal_reward',total_rewards,
                      '\nrewards',env.rewards,
                      '\nbuckets', [env.board.agents[i].bucket for i in env.agents]
                     )
                print('kill_list',env.kill_list,'term',e_t, 'buckets',[i.bucket for i in env.board.agents])
                #if sum(env.curr_rewards.values()) <0:
                #    input()
        else:
            print(env.board.territories)
            print('done',env.terminations.values(),env.curr_rewards)
            break
step


#need to debug .... so the action is valid... but there is no change
#so either the phase is not changing and the agent is not changing... why??


[[2 1.0]
 [2 1.0]
 [2 4.0]
 [2 4.0]
 [2 9.0]
 [2 1.0]
 [2 1.0]
 [2 4.0]
 [2 1.0]
 [2 1.0]]
done dict_values([True, True, True, True]) {1: -100, 2: 100, 3: -100, 4: -100}


3578